In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os

def findFiles(path): return glob.glob(path)

print(findFiles('data/names/*.txt'))

import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)
### all_letters = a~z+A~Z+ .,;' (記号)
print(f"all_letters   type:{type(all_letters)}, len: {len(all_letters)}")


# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicodeToAscii('Ślusàrski'))

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for index,filename in enumerate(findFiles('data/names/*.txt')):
    ### basename  - *.txt (メインのファイル名)
    ### splittext - 拡張子を分割 [0]をとることで拡張子削除
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    if index == 0:
        print(category)
        print(lines)
    category_lines[category] = lines

n_categories = len(all_categories)

['data/names/Spanish.txt', 'data/names/Arabic.txt', 'data/names/French.txt', 'data/names/Greek.txt', 'data/names/Dutch.txt', 'data/names/German.txt', 'data/names/Scottish.txt', 'data/names/English.txt', 'data/names/Czech.txt', 'data/names/Chinese.txt', 'data/names/Korean.txt', 'data/names/Polish.txt', 'data/names/Italian.txt', 'data/names/Russian.txt', 'data/names/Irish.txt', 'data/names/Vietnamese.txt', 'data/names/Portuguese.txt', 'data/names/Japanese.txt']
all_letters   type:<class 'str'>, len: 57
Slusarski
Spanish
['Abana', 'Abano', 'Abarca', 'Abaroa', 'Abascal', 'Abasolo', 'Abel', 'Abello', 'Aberquero', 'Abreu', 'Acosta', 'Agramunt', 'Aiza', 'Alamilla', 'Albert', 'Albuquerque', 'Aldana', 'Alfaro', 'Alvarado', 'Alvarez', 'Alves', 'Amador', 'Andreu', 'Antunez', 'Aqua', 'Aquino', 'Araujo', 'Araullo', 'Araya', 'Arce', 'Arechavaleta', 'Arena', 'Aritza', 'Armando', 'Arreola', 'Arriola', 'Asis', 'Asturias', 'Avana', 'Azarola', 'Banderas', 'Barros', 'Basurto', 'Bautista', 'Bello', 'Belmon

In [3]:
print(n_categories)

print(all_categories[:3])

print(category_lines['Italian'][:5])

18
['Spanish', 'Arabic', 'French']
['Abandonato', 'Abatangelo', 'Abatantuono', 'Abate', 'Abategiovanni']


In [4]:
import torch

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(letterToTensor('J'))

print(lineToTensor('Jones').size())

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]])
torch.Size([5, 1, 57])


In [5]:
a = torch.rand((2,3))
print(a,a.shape)
b = torch.arange(8).reshape(2,4)
print(b,b.shape)

c = torch.cat((a,b),1)
print(c)

tensor([[0.9079, 0.8682, 0.1551],
        [0.8136, 0.1870, 0.1724]]) torch.Size([2, 3])
tensor([[0, 1, 2, 3],
        [4, 5, 6, 7]]) torch.Size([2, 4])
tensor([[0.9079, 0.8682, 0.1551, 0.0000, 1.0000, 2.0000, 3.0000],
        [0.8136, 0.1870, 0.1724, 4.0000, 5.0000, 6.0000, 7.0000]])


In [6]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

In [20]:
input = lineToTensor('Albert')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input[0], hidden)
print(output,output.shape)

print(output.topk(3))

tensor([[-2.9844, -2.9482, -2.8458, -2.8929, -2.8659, -2.9100, -2.9781, -2.8027,
         -2.9244, -2.9009, -2.9635, -2.9224, -2.8271, -2.8009, -2.8325, -2.9573,
         -2.9488, -2.7603]], grad_fn=<LogSoftmaxBackward0>) torch.Size([1, 18])
torch.return_types.topk(
values=tensor([[-2.7603, -2.8009, -2.8027]], grad_fn=<TopkBackward0>),
indices=tensor([[17, 13,  7]]))
-2.7602930068969727


In [12]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    print(top_n,top_i)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i

print(categoryFromOutput(output))

tensor([[-2.7603]], grad_fn=<TopkBackward0>) tensor([[17]])
('Japanese', 17)


In [21]:
import random
print(random.randint(0,4))

1


In [26]:
import random

### lはリストを受け取る．つまり返り値はその要素
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    category = randomChoice(all_categories)
    print(category,type(category))
    line = randomChoice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

Italian <class 'str'>
category = Italian / line = Garfagnini
Italian <class 'str'>
category = Italian / line = Cremonesi
French <class 'str'>
category = French / line = Vincent
Vietnamese <class 'str'>
category = Vietnamese / line = Van
German <class 'str'>
category = German / line = Martin
English <class 'str'>
category = English / line = Myers
Russian <class 'str'>
category = Russian / line = Bakhmutski
Vietnamese <class 'str'>
category = Vietnamese / line = Tong
Scottish <class 'str'>
category = Scottish / line = Robertson
Korean <class 'str'>
category = Korean / line = Hung


In [ ]:
#